# Analysons la qualité de l'air

In [60]:
import pandas as pd
import numpy as np 
import re

data = pd.read_csv('openaq.csv', sep=';')
data

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,DE,Manfred Lotz,"Cottbus, W.-Külz-Str.","51.75413,14.325549",PM2.5,EEA Germany,µg/m³,21.7,2016-12-21T12:00:00+01:00,Germany
1,ES,Castellón/Castelló,ES1969A,"39.8236111095,-0.24444444",NO2,EEA Spain,µg/m³,1.0,2021-07-13T08:00:00+02:00,Spain
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.0,2021-08-06T11:00:00+02:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.0,2021-08-06T11:00:00+02:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.0,2021-08-06T11:00:00+02:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40446,US,BERNALILLO,WESTSIDE - 9 Mile,"35.064098,-106.761292",PM10,AirNow,µg/m³,10.0,2020-07-17T14:00:00+02:00,United States
40447,US,Asheville,Skyland DRR,"35.481861,-82.509861",SO2,AirNow,ppm,0.0,2020-07-02T12:00:00+02:00,United States
40448,US,San Antonio,San Antonio IH35 C10,"29.5294,-98.39139",PM2.5,AirNow,µg/m³,6.0,2020-06-26T01:00:00+02:00,United States
40449,US,Cleveland-Elyria-Mentor,Cleveland Near Road,"41.4409,-81.4949",PM2.5,AirNow,µg/m³,15.3,2020-05-26T16:00:00+02:00,United States


### Supprimer les données qui ne concernent pas la journée du 06-08-2021.

In [61]:
data['Last Updated']=pd.to_datetime(data['Last Updated'], utc=True).dt.tz_convert(None)
data = data.loc[(data['Last Updated'] >= '2021-08-06 00:00:00') & (data['Last Updated'] < '2021-08-07 00:00:00')]
data

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06 09:00:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06 09:00:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06 09:00:00,Spain
5,ES,Valencia/València,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06 09:00:00,Spain
6,ES,Valencia/València,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06 09:00:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06 12:00:00,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06 11:00:00,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06 11:00:00,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06 11:00:00,Thailand


### Supprimer les données qui sont illisibles et pour lesquelles on ne comprend pas la ville concernée. 

In [62]:
data = data.dropna(subset = ['City'])
data

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06 09:00:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06 09:00:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06 09:00:00,Spain
5,ES,Valencia/València,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06 09:00:00,Spain
6,ES,Valencia/València,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06 09:00:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06 12:00:00,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06 11:00:00,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06 11:00:00,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06 11:00:00,Thailand


In [63]:
indexNames = data[data['Source Name'] == 'ChinaAQIData'].index
data.drop(indexNames , inplace=True)
data

C:\Users\Donovan\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06 09:00:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06 09:00:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06 09:00:00,Spain
5,ES,Valencia/València,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06 09:00:00,Spain
6,ES,Valencia/València,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06 09:00:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06 12:00:00,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06 11:00:00,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06 11:00:00,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06 11:00:00,Thailand


### Pour chaque polluant, vérifier qu'on n'a qu'une seule mesure utilisée. Autrement, supprimer les données qui correspondent à la mesure minoritaire ou moins cohérente.

In [64]:
indexNames = data[data['City'] == '037'].index
data.drop(indexNames , inplace=True)
data

C:\Users\Donovan\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06 09:00:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06 09:00:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06 09:00:00,Spain
5,ES,Valencia/València,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06 09:00:00,Spain
6,ES,Valencia/València,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06 09:00:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06 12:00:00,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06 11:00:00,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06 11:00:00,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06 11:00:00,Thailand


In [65]:
data.pivot_table(index=['City','Location','Pollutant', 'Source Name'], values='Value', aggfunc=["count"]).to_csv('tcd2.csv')

In [66]:
data.to_csv('exo2.csv')

In [67]:
à corriger :
    City	Location	Pollutant	Source Name	Value
Bakersfield	Bakersfield - Califo	NO2	AirNow	2
Bakersfield	Bakersfield - Califo	O3	AirNow	2
Bakersfield	Bakersfield - Califo	PM2.5	AirNow	2
Bellingham	Custer-Loomis (SO-AQ	O3	AirNow	2
Bexley	Bexley - Belvedere West FDMS	PM10	London Air Quality Network	2
Bishop	Coso Junction	PM10	AirNow	2
Bishop	NCORE - Bishop	CO	AirNow	2
Bishop	NCORE - Bishop	O3	AirNow	2
Bishop	NCORE - Bishop	PM10	AirNow	2
Bishop	NCORE - Bishop	PM2.5	AirNow	2
Buffalo City Metro	Mobile	CO	South Africa	2
Sarasota-Bradenton-Venice	Lido Park			
Buffalo City Metro	Mobile	NO2	South Africa	2
Buffalo City Metro	Mobile	SO2	South Africa	2
Dartford	Dartford Roadside 2 - Town Centre	NO2	London Air Quality Network	2
Decatur	DECATUR	O3	AirNow	2
Decatur	DECATUR	PM2.5	AirNow	2
eThekwini Metro	Settlers	PM10	South Africa	2
eThekwini Metro	Settlers	PM2.5	South Africa	2
eThekwini Metro	Settlers	SO2	South Africa	2
eThekwini Metro	Wentworth Reservior	PM10	South Africa	2
eThekwini Metro	Wentworth Reservior	PM2.5	South Africa	2
eThekwini Metro	Wentworth Reservior	SO2	South Africa	2
Eugene-Springfield	Oakridge - Willamett	PM2.5	AirNow	2
Eugene-Springfield	Springfield City Hal	PM2.5	AirNow	2
Gert Sibande	Bosjesspruit	NO2	South Africa	2
Gert Sibande	Bosjesspruit	O3	South Africa	2
Gert Sibande	Bosjesspruit	PM10	South Africa	2
Gert Sibande	Bosjesspruit	PM2.5	South Africa	2
Gert Sibande	Bosjesspruit	SO2	South Africa	2
Gert Sibande	Club	O3	South Africa	2
Gert Sibande	Club	PM10	South Africa	2
Gert Sibande	Club	PM2.5	South Africa	2
Gert Sibande	Club	SO2	South Africa	2
Gert Sibande	Embalenhle	O3	South Africa	2
Gert Sibande	Embalenhle	PM10	South Africa	2
Gert Sibande	Embalenhle	PM2.5	South Africa	2
Gert Sibande	Embalenhle	SO2	South Africa	2
Gert Sibande	Lebohang	NO2	South Africa	2
Gert Sibande	Lebohang	O3	South Africa	2
Gert Sibande	Lebohang	SO2	South Africa	2
Gert Sibande	eMbalenhle South	NO2	South Africa	2
Gert Sibande	eMbalenhle South	PM10	South Africa	2
Gert Sibande	eMbalenhle South	PM2.5	South Africa	2
Gettysburg	Arendtsville	O3	AirNow	2
Iowa City	Iowa City, Hoover Sc	PM2.5	AirNow	2
King Cetshwayo	Scorpio - RBCAA	SO2	South Africa	2
LillestrÃ¸m	Vollaparken Ã¸st	NO2	Norway	4
LillestrÃ¸m	Vollaparken Ã¸st	PM10	Norway	4
LillestrÃ¸m	Vollaparken Ã¸st	PM2.5	Norway	4
LJ ViÄ	LJ ViÄ	PM10	Slovenian Environment Agency	2
LOS ANGELES	710 Near Road	NO2	AirNow	3
Oxnard-Thousand Oaks-Ventura	Piru - Pacific	O3	AirNow	2
Oxnard-Thousand Oaks-Ventura	Piru - Pacific	PM2.5	AirNow	2
Oxnard-Thousand Oaks-Ventura	Thousand Oaks - Moor	O3	AirNow	2
Oxnard-Thousand Oaks-Ventura	Thousand Oaks - Moor	PM2.5	AirNow	2
PEND OREILLE	Newport-Calispel	PM2.5	AirNow	2
Pendleton-Hermiston	Pendleton - McKay Cr	PM2.5	AirNow	2
Portland-Vancouver-Beaverton	Hillsboro - Hare Fie	PM2.5	AirNow	2
RibeirÃ£o Preto	RibeirÃ£o Preto	CO	Sao Paulo CETESB	2
RibeirÃ£o Preto	RibeirÃ£o Preto	O3	Sao Paulo CETESB	2
RibeirÃ£o Preto	RibeirÃ£o Preto	PM10	Sao Paulo CETESB	2
RibeirÃ£o Preto	RibeirÃ£o Preto	PM2.5	Sao Paulo CETESB	2
Riverside-San Bernardino-Ontario	Riverside - Rubidoux	CO	AirNow	2
Riverside-San Bernardino-Ontario	Riverside - Rubidoux	NO2	AirNow	2
Riverside-San Bernardino-Ontario	Riverside - Rubidoux	O3	AirNow	2
Riverside-San Bernardino-Ontario	Riverside - Rubidoux	PM10	AirNow	2
Riverside-San Bernardino-Ontario	Riverside - Rubidoux	PM2.5	AirNow	2
Riverside-San Bernardino-Ontario	Riverside - Rubidoux	SO2	AirNow	2
Rochester	Rochester	O3	AirNow	2
Rochester	Rochester	PM2.5	AirNow	2
SAN LUIS OBISPO	Nipomo Mesa CDF	PM10	AirNow	2
Deltona-Daytona Beach-Ormond Beach	Blind Services			
Deltona-Daytona Beach-Ormond Beach	Blind Services			
SAN LUIS OBISPO	Nipomo Mesa CDF	PM2.5	AirNow	2
San Luis Obispo-Paso Robles	Nipomo RP	O3	AirNow	2
San Luis Obispo-Paso Robles	Nipomo RP	PM10	AirNow	2
San Luis Obispo-Paso Robles	SLO Roberto	PM10	AirNow	3
San Luis Obispo-Paso Robles	SLO Roberto	PM2.5	AirNow	3
Sierpc	Sierpc-WiosnyLudÃ³wZSznr2	PM10	GIOS	2
Sierpc	Sierpc-WiosnyLudÃ³wZSznr2	PM2.5	GIOS	2
Sopot	Sopot Bitwy Pod PÅ‚owcami	CO	GIOS	2
Sopot	Sopot Bitwy Pod PÅ‚owcami	NO2	GIOS	2
Sopot	Sopot Bitwy Pod PÅ‚owcami	PM10	GIOS	2
Sopot	Sopot Bitwy Pod PÅ‚owcami	PM2.5	GIOS	2
Sopot	Sopot Bitwy Pod PÅ‚owcami	SO2	GIOS	2
Spokane	Spokane-Augusta Ave	PM10	AirNow	2
Spokane	Spokane-Augusta Ave	PM2.5	AirNow	2
Visalia-Porterville	Sequoia and Kings Ca	O3	AirNow	2
Windsor and Maidenhead	Windsor and Maidenhead - Aldebury Road	NO2	London Air Quality Network	2

SyntaxError: invalid syntax (Temp/ipykernel_19020/373079571.py, line 1)